# 디스크 컨트롤러

각 작업에 대해 작업이 요청되는 시점, 작업의 소요시간 을 담은 2차원 배열 jobs가 매개변수로 주어질 때, 작업의 요청부터 종료까지 걸린 시간의 평균을 가장 줄이는 방법으로 처리하면 평균이 얼마가 되는지 return 하도록 solution 함수를 작성해주세요. (단, 소수점 이하의 수는 버립니다)

### 제한 사항
- jobs의 길이는 1 이상 500 이하입니다.
- jobs의 각 행은 하나의 작업에 대한 작업이 요청되는 시점, 작업의 소요시간 입니다.
- 각 작업에 대해 작업이 요청되는 시간은 0 이상 1,000 이하입니다.
- 각 작업에 대해 작업의 소요시간은 1 이상 1,000 이하입니다.
- 하드디스크가 작업을 수행하고 있지 않을 때에는 먼저 요청이 들어온 작업부터 처리합니다.

In [3]:
# timing decorator function
from functools import wraps
from time import time

def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        print ('func:%r args:[%r, %r] took: %2.4f sec' % \
          (f.__name__, args, kw, te-ts))
        return result
    return wrap

### 생각 1

1. 먼저 들어온 순서대로 처리하는 방법으로 계산해보고
2. 순서를 변경하여 계산해본다. => 이전 작업이 끝나는 시점과 가장 가까운 요청 시간으로 들어온 작업을 시작시킨다 (timer 값을 지정하여, 타임라인을 확인.)  
    1. timer 값과 가장 가까운 시기에 요청이 들어온 작업들을 우선적으로 시작시킨다.
    2. 우선 timer 보다 작거나 같은 값들을 대상으로 찾아보고
    3. 없다면, timer 보다 큰 값들을 대상으로 찾아본다.
    4. 해당 작업이 찾아지면, 해당 작업을 시작한다.
    5. timer를 해당 작업이 끝나는 시점으로 이동시킨다. (timer + 작업 소요시간)
    6. 작업 소요시간 목록에 (timer - 작업 요청시간) 값을 추가한다.
3. 두 값을 비교한다


# 시도 1 : 실패 (test case 3개 통과)

In [54]:
import heapq

@timing
def solution(jobs):
    answer = 0
    
    # 순차적으로 계산하는 방법
    sequential_timer = 0
    sequential_times = []    
    sequential_avg = 0   
    # O(N) 
    for job in jobs:
        request_time = job[0]
        process_time = job[1]
        
        diff = job[0] - sequential_timer

        # 이전 작업을 완료한 시간보다 이후에 다음 작업이 시작된다면, 대기 시간만큼 더함.        
        if diff > 0 :
            sequential_timer += diff
        # 작업 소요시간 더해줌        
        sequential_timer += process_time

        sequential_times.append(sequential_timer-request_time)
    sequential_avg = round(sum(sequential_times)/ len(sequential_times))

    # print(round(sequential_avg))

    # 순서를 변경하여 계산해보기
    timer = 0
    times = []
    avg = 0
    jobs_copy = jobs.copy()     
    while len(times) < len(jobs):
        # 현재 timer 값보다 작거나 작은 값을 찾는다.       
        under_timer = [job for job in jobs_copy if job[0]<=timer]
        under_timer.sort(reverse=True)   

        # print(under_timer)
        if len(under_timer)>0:
            job = under_timer[0]
            jobs_copy.remove(job)
            # timer를 해당 작업이 끝나는 시점으로 이동시킨다. (timer + 작업 소요시간)
            timer += job[1]
            # 작업 소요시간 목록에 (timer - 작업 요청시간) 값을 추가한다.
            times.append(timer - job[0])
            
            # print(job, timer, times)
        else :
            # 현재 timer 값보다 큰 값을 찾는다.
            upper_timer = [job for job in jobs_copy if job[0]>timer]
            upper_timer.sort() 

            #print(upper_timer)
            
            job = upper_timer[0]
            jobs_copy.remove(job)
            
            # 작업 시작 전까지 기다렸던 시간을 더해준다.
            diff = job[0] - timer
            timer += diff
            # timer를 해당 작업이 끝나는 시점으로 이동시킨다.
            timer += job[1]
             # 작업 소요시간 목록에 (timer - 작업 요청시간) 값을 추가한다.
            times.append(timer - job[1])
            #print(job, timer, times)

    avg = round(sum(times)/ len(times))

    #print(sequential_avg, avg)
    
    answer = avg

    if sequential_avg < avg:
        answer = sequential_avg        
            
    return answer

In [55]:
solution([[0, 3], [1, 9], [2, 6]])

func:'solution' args:[([[0, 3], [1, 9], [2, 6]],), {}] took: 0.0000 sec


9